<h1>Segmenting and Clustering Neighborhoods in Toronto

<b><i> A single Notebook is created for all the three parts </b>

<h2>Part 1: Scraping Postal Codes from Wikipidea

In [10]:
import pandas as pd  # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files
import requests # library to handle requests

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [135]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

wiki_table = soup.body.table.tbody
print(wiki_table)

<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:80%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:80%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:80%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:80%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:80%;"><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a><br/>(<a href="/wiki/Regent_Park" title="Regent 

In [187]:
postal_code_data = []
for row in wiki_table.find_all('tr'):
    
    for cell in row.find_all('td'):
        postal_code = cell.p.b.text # get the postal code
        neighborhood_data = cell.span.text  # get the rest of the data in the cell
        borough = neighborhood_data.split('(')[0] # get the borough in the cell

        if borough == 'Not assigned': # check wheather cell has borough or not
            continue
        else: #process data and add to the list
            if len(neighborhood_data.split('('))>1:
                neighborhoods = neighborhood_data.split('(')[1]
                neighborhoods = neighborhoods.replace(')',' ')
                neighborhoods = ', '.join([neighbor.strip() for neighbor in neighborhoods.split('/')]) 
            else:
                borough= borough.strip('\n')
                neighborhoods = borough
            
        postal_code_data.append([postal_code,borough,neighborhoods])


In [201]:
#convert list into dataFrame
columns = ['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(postal_code_data,columns = columns)
df.sort_values(by='PostalCode',inplace=True) #sort data Frame
df.reset_index(drop=True,inplace=True) #rest its index
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [202]:
df.shape # print number of rows of dataframe

(103, 3)

<h2> Part 2: Extend the dataframe built above with the coordinates details

In [203]:
geo_data_toronto = pd.read_csv('http://cocl.us/Geospatial_data') #read csv file that contains coordinates.
geo_data_toronto.columns = ['PostalCode','Latitude','Longitude']
geo_data_toronto.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


<b>Merging the dataframe with the neighborhood details dataframe created in Part 1 to create the final dataframe

In [204]:
df_toronto = pd.merge(df,geo_data_toronto,on = 'PostalCode')

In [205]:
df_toronto.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


<h2>Part 3: Explore and cluster the neighborhoods in Toronto

In [206]:
#Let's simplify the above dataset and cluster only the boroughs containing Torontto. So let's slice the original dataframe and create a new dataframe.
toronto_borough = df_toronto[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
4,M4M,East Toronto,Studio District,43.659526,-79.340923


In [207]:
print('We have a total of %d Postal codes with Toronto' % toronto_borough.shape[0])

We have a total of 39 Postal codes with Toronto


In [242]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [209]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [213]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [214]:
toronto_venues = getNearbyVenues(names=toronto_borough['Neighborhood'],
                                 latitudes=toronto_borough['Latitude'],
                                 longitudes=toronto_borough['Longitude']
                                )

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [215]:
print(toronto_venues.shape)
toronto_venues.head(10)

(1688, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth East,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
5,The Danforth East,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store
6,The Danforth East,43.685347,-79.338106,The Red Rocket,43.688048,-79.333274,Coffee Shop
7,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
8,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
9,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant


In [216]:
print('The number of popular venues in each borough are as follows:')
toronto_venues.groupby('Neighborhood').count()

The number of popular venues in each borough are as follows:


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,83,83,83,83,83,83
Christie,18,18,18,18,18,18
Church and Wellesley,87,87,87,87,87,87
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,38,38,38,38,38,38
Davisville North,7,7,7,7,7,7


In [217]:
print('There are %d unique categories of venues in Toronto Boroughs' % len(toronto_venues['Venue Category'].unique()))
toronto_venues['Venue Category'].unique()

There are 240 unique categories of venues in Toronto Boroughs


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Park',
       'Convenience Store', 'Coffee Shop', 'Greek Restaurant',
       'Cosmetics Shop', 'Italian Restaurant', 'Ice Cream Shop',
       'Yoga Studio', 'Brewery', 'Fruit & Vegetable Store', 'Pizza Place',
       'Bookstore', 'Restaurant', 'Dessert Shop', 'Juice Bar',
       'Bubble Tea Shop', 'Spa', 'Diner', 'Grocery Store',
       'Furniture / Home Store', 'Café', 'Bakery', 'Caribbean Restaurant',
       'American Restaurant', 'Frozen Yogurt Shop', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Pet Store', 'Steakhouse', 'Burrito Place', 'Movie Theater',
       'Sandwich Place', 'Board Shop', 'Fish Market', 'Gay Bar',
       'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Thai Restaurant', 'Seafood Restaurant',
       'Stationery Store', 'Coworking Space', 'Wine Bar', 'Bar',
       'Gym / Fitness Center', 'Latin American Restaurant', '

<h3>Analyze each neighborhood

In [222]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = toronto_onehot.rename(columns={"Neighborhood": "Neighborhod"})
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth East,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [223]:
#And let's examine the new dataframe size
toronto_onehot.shape

(1688, 241)

In [224]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby("Neighborhood").mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
3,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.012048,0.0,...,0.0,0.0,0.012048,0.0,0.0,0.012048,0.0,0.0,0.0,0.012048
4,Christie,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000


In [225]:
#Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1  Farmers Market  0.04
2            Café  0.04
3          Bakery  0.04
4      Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0           Nightclub  0.08
1      Breakfast Spot  0.08
2         Coffee Shop  0.08
3                Café  0.08
4  Italian Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4     Boat or Ferry  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1   Italian Restaurant  0.05
2       Sandwich Place  0.04
3         Burger Joint  0.04
4  Japanese Restaurant  0.04


----Christie----
           venue  freq
0  Grocery Store  0.22
1           Café  0.17
2           Park  0.11
3     Baby Store  0.06
4  

<b>Let's put that into a pandas dataframe

In [226]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [227]:
#Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Cheese Shop,Café,Restaurant,Farmers Market,Beer Bar,Seafood Restaurant,Museum
1,"Brockton, Parkdale Village, Exhibition Place",Nightclub,Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Bakery,Performing Arts Venue,Pet Store,Climbing Gym,Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Plane,Airport,Airport Food Court,Airport Gate,Sculpture Garden,Harbor / Marina,Boutique
3,Central Bay Street,Coffee Shop,Italian Restaurant,Japanese Restaurant,Burger Joint,Ice Cream Shop,Sandwich Place,Gym / Fitness Center,Department Store,Restaurant,Middle Eastern Restaurant
4,Christie,Grocery Store,Café,Park,Gas Station,Italian Restaurant,Restaurant,Diner,Baby Store,Athletics & Sports,Nightclub


<h3>Clustering the neighborhood using k-means clustering

In [228]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [229]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
neighborhoods_venues_sorted.iloc[0:0]
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted
toronto_merged = toronto_borough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged= toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhod,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
1,M4J,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,1,Coffee Shop,Convenience Store,Park,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
3,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Coffee Shop,Movie Theater,Sandwich Place,Board Shop,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
4,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Yoga Studio,Fish Market,Pet Store,Park


In [230]:
toronto_merged = toronto_merged.drop('PostalCode', 1)
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhod,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio
1,East YorkEast Toronto,The Danforth East,43.685347,-79.338106,1,Coffee Shop,Convenience Store,Park,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
3,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,1,Park,Coffee Shop,Movie Theater,Sandwich Place,Board Shop,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
4,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Yoga Studio,Fish Market,Pet Store,Park
5,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
6,Central Toronto,Davisville North,43.712751,-79.390197,1,Gym,Department Store,Park,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Dumpling Restaurant,Doner Restaurant,Donut Shop
7,Central Toronto,North Toronto West,43.715383,-79.405678,1,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Café,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Mexican Restaurant
8,Central Toronto,Davisville,43.704324,-79.388790,1,Pizza Place,Sandwich Place,Dessert Shop,Gym,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Asian Restaurant,Japanese Restaurant
9,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,4,Restaurant,Park,Tennis Court,Playground,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Department Store,Donut Shop


In [231]:
#Finally, let's visualize the resulting clusters
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [232]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
        
map_clusters

<h3>Examine the clusters

<h4>Cluster 1

In [237]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
24,Forest Hill North & West,Jewelry Store,Bus Line,Park,Sushi Restaurant,Trail,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio


<h4>Cluster 2

In [238]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,The Danforth East,Coffee Shop,Convenience Store,Park,Yoga Studio,Discount Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
2,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
3,"India Bazaar, The Beaches West",Park,Coffee Shop,Movie Theater,Sandwich Place,Board Shop,Burrito Place,Fast Food Restaurant,Fish & Chips Shop,Steakhouse,Italian Restaurant
4,Studio District,Café,Coffee Shop,American Restaurant,Bakery,Brewery,Italian Restaurant,Yoga Studio,Fish Market,Pet Store,Park
6,Davisville North,Gym,Department Store,Park,Breakfast Spot,Sandwich Place,Hotel,Food & Drink Shop,Dumpling Restaurant,Doner Restaurant,Donut Shop
7,North Toronto West,Clothing Store,Coffee Shop,Yoga Studio,Sporting Goods Shop,Café,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Mexican Restaurant
8,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Gym,Sushi Restaurant,Coffee Shop,Café,Italian Restaurant,Asian Restaurant,Japanese Restaurant
10,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Pizza Place,Fried Chicken Joint,Liquor Store,Sports Bar,Restaurant,Supermarket,Sushi Restaurant,Bank
12,"St. James Town, Cabbagetown",Coffee Shop,Pharmacy,Pet Store,Pub,Restaurant,Bakery,Pizza Place,Café,Park,Italian Restaurant
13,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Gastropub,Hotel,Pub,Bubble Tea Shop,Men's Store


<h4>Cluster 3

In [239]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Neighborhod,Trail,Health Food Store,Pub,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Yoga Studio


<h4>Cluster 4

In [240]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Roselawn,Garden,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


<h4>Cluster 5

In [241]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,"Moore Park, Summerhill East",Restaurant,Park,Tennis Court,Playground,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Department Store,Donut Shop
11,Rosedale,Park,Playground,Trail,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
